In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
## export
import torch
from torch import nn
import torch.nn.functional as F


In [7]:

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
#         self.convs = 
    def forward(self):
        pass
    
class UberResBlock(nn.Module):
    pass


class ResBlock(nn.Module):
    NUM_OF_CONVS = 2 
    def __init__(self,in_channels: int, out_channels: int, kernel_size: int,
                 stride: int, **kwargs):
        super().__init__()
        layers = []
        for i in range(self.NUM_OF_CONVS):
            layers.append(nn.Conv2d(in_channels,out_channels,kernel_size, **kwargs))
            if i==0:
                layers.append(nn.ReLU())

        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x) + x
    
        

In [8]:
x = ResBlock(1,1,3,2)
print(x)

ResBlock(
  (model): Sequential(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  )
)
